# Practice Session Insights

In [1]:
# Setup
from db_util import *
from IPython.display import display
import pandas as pd
import numpy as np

cnxnObject = DBConnection()
cnxn = cnxnObject.connection
cnxnObject.sample_query()

Microsoft SQL Server 2016 (SP1-CU15-GDR) (KB4505221) - 13.0.4604.0 (X64) 
	Jun 15 2019 07:56:34 
	Copyright (c) Microsoft Corporation
	Developer Edition (64-bit) on Windows Server 2016 Datacenter 10.0 <X64> (Build 14393: ) (Hypervisor)



---

## Session Analytics

We are interested in finding metrics for UserTestSession in which a student attempts questions from a topic. We define the metrics as follows:

* Session Quantity
    * No. Of Questions vs. global avg in topic / chapter / subject
    * Time vs. global avg in topic / chapter / subject
* Session Quality
    * Accuracy
        * vs. Global average
        * vs. Student subject average
        * vs. Student previous sessions
    * Time
        * vs. Global average
        * vs. Student subject average
        * vs. Student previous sessions

### Metrics 

To compare the student's performance we introduce 
* Session metrics
    * Pseudo
    * Student previous
* Global metrics
    * Topic
    * Chapter
    * Subject
* Student metrics
    * Topic
    * Chapter
    * Subject

In [2]:
# Lets fetch the session
UserId = 46733  # 31283, 46733, 62525, 46965
dateOnwards = "01-01-2021"
userSessions = get_sessions(
    userId=UserId, dateOnwards=dateOnwards, connection=cnxn)

# Filter for greater than threshold questions.
minQustions = 5
userSessions = userSessions[userSessions["_TotalQuestions"] > minQustions]
print("User Sessions:")
display(userSessions)


User Sessions:


,UserTestSessionId,CourseId,SubjectId,TopicId,DifficultyLevelId,_TotalQuestions,_TotalAttempted,_TotalCorrect
0,262867,1,1,8,1,57,56,26
2,264253,1,1,8,1,7,7,3
3,268692,1,1,4,1,23,22,13
5,279888,1,1,6,1,22,17,12
6,288482,1,1,1,1,20,18,15
8,288500,1,1,1,2,32,29,19
9,308111,1,1,4,1,23,23,12
10,308457,1,1,4,1,21,21,13
12,323105,1,1,4,1,7,7,2
13,323123,1,1,7,1,7,6,4


In [3]:
# Sample session
UserSessionId = userSessions["UserTestSessionId"].iloc[48]
pseudoQuestionMetrics, pseudoSessionMetrics = get_pseudo_session_metrics(UserSessionId,dateOnwards,cnxn)
display_nonzero(pseudoQuestionMetrics)
display_nonzero(pseudoSessionMetrics)


,QuestionId,TimesAppeared,AttemptedPercentage,Accuracy,AvgTimeTaken,StdTimeTaken,AttemptedSecondPercentage,AccuracySecond
0,105092,87,89.655172,41.37931,49.62069,46.218681,92.857143,38.095238
1,105090,76,94.736842,69.736842,19.0,19.306394,94.736842,63.157895
2,105081,80,92.5,38.75,75.775,81.862991,93.023256,37.209302
3,105080,38,84.210526,39.473684,44.394737,48.208724,94.117647,58.823529
4,105078,70,85.714286,55.714286,46.657143,96.144368,95.238095,23.809524
5,105076,67,92.537313,67.164179,17.268657,17.557226,94.117647,58.823529
6,105074,57,96.491228,61.403509,21.754386,34.895976,95.0,50.0
7,105073,76,86.842105,35.526316,38.644737,46.598426,89.74359,38.461538
8,105072,75,85.333333,61.333333,51.653333,55.130752,94.444444,38.888889
9,105071,61,91.803279,52.459016,32.770492,45.775788,91.666667,54.166667


,Mean,Std
NoOfQuestions,24,0
AttemptedPercentage,87.480356,4.357116
Accuracy,49.816658,11.396479
TimeTaken,1302.082463,286.372051
AttemptedSecondPercentage,92.694177,3.406616
AccuracySecond,45.320585,12.008087


In [4]:
sessionMetrics = get_session_metrics(UserSessionId,cnxn)
display(userSessions)
topicMetrics, _ = get_global_topic_metrics(int(userSessions[userSessions["UserTestSessionId"]==UserSessionId]["TopicId"]),dateOnwards,cnxn)
display(sessionMetrics)
display(topicMetrics)
df = pseudoSessionMetrics.copy()
df.rename(columns={"Mean":"PseudoMean","Std":"PseudoStd"})
df["Student"] = sessionMetrics
display_nonzero(df)

,UserTestSessionId,CourseId,SubjectId,TopicId,DifficultyLevelId,_TotalQuestions,_TotalAttempted,_TotalCorrect
0,262867,1,1,8,1,57,56,26
2,264253,1,1,8,1,7,7,3
3,268692,1,1,4,1,23,22,13
5,279888,1,1,6,1,22,17,12
6,288482,1,1,1,1,20,18,15
8,288500,1,1,1,2,32,29,19
9,308111,1,1,4,1,23,23,12
10,308457,1,1,4,1,21,21,13
12,323105,1,1,4,1,7,7,2
13,323123,1,1,7,1,7,6,4


d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:418: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AttemptedSecondPercentage"][idx]    = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"].notnull()) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:419: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AccuracySecond"][idx]               = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"]) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))


Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []


d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:418: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AttemptedSecondPercentage"][idx]    = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"].notnull()) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:419: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AccuracySecond"][idx]               = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"]) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:418: RuntimeWarning: divide by zero encountered in double_scalars
  topicMetrics["AttemptedSecondPercentage"][idx]    = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"].notnull()) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:419: RuntimeWarning: divide by zero encountered in doubl

Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []


...

Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []
Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []
Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []
Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []
Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen, IsCorrectlyAnswered, TimeTakenTillSubmission, IsFullSolutionSeen, IsCorrectlyAnsweredSecondTime, MarkedForRevision]
Index: []
Empty DataFrame
Columns: [QuestionId, IsAttempted, IsKSCSeen

d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:418: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AttemptedSecondPercentage"][idx]    = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"].notnull()) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:419: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AccuracySecond"][idx]               = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"]) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:418: RuntimeWarning: invalid value encountered in double_scalars
  topicMetrics["AttemptedSecondPercentage"][idx]    = 100.0 * np.sum(qData["IsCorrectlyAnsweredSecondTime"].notnull()) / np.sum(~(qData["IsCorrectlyAnswered"].to_numpy(dtype=bool)))
d:\Work\SpeEdLabs\SpeEdLabs-Analytics\src\db_util.py:419: RuntimeWarning: invalid value encountered in double_

NoOfQuestions                    24.000000
AttemptedPercentage              91.666667
KSCViewedPercentage               0.000000
Accuracy                         50.000000
TimeTaken                      1180.000000
FullSolutionSeenPercentage        0.000000
AttemptedSecondPercentage        83.333333
AccuracySecond                   66.666667
MarkedForRevisionPercentage       0.000000
dtype: float64

,Mean,Std
NoOfQuestions,32.43508,28.888845
AttemptedPercentage,NaN,NaN
KSCViewedPercentage,NaN,NaN
Accuracy,NaN,NaN
TimeTaken,235851.399428,5231.029192
FullSolutionSeenPercentage,NaN,NaN
AttemptedSecondPercentage,NaN,NaN
AccuracySecond,NaN,NaN
MarkedForRevisionPercentage,NaN,NaN


,Mean,Std,Student
NoOfQuestions,24,0,24.000000
AttemptedPercentage,87.480356,4.357116,91.666667
Accuracy,49.816658,11.396479,50.000000
TimeTaken,1302.082463,286.372051,1180.000000
AttemptedSecondPercentage,92.694177,3.406616,83.333333
AccuracySecond,45.320585,12.008087,66.666667


In [5]:
testUserId = 404

# Query the TestUserId Table
if "Querty the TestUserID Table":
    query = """
    SELECT  
    [TestId]
    ,[UserId]
    ,[StartedOn]
    ,[CompletedOn]
    ,[Status]
    FROM    
    [speedlabs-anon].[dbo].[TestUser]
    WHERE
    [TestUserId] = {x}
    """.format(x=testUserId)
    data = pd.read_sql(query, connection.connection)

    testSummary = pd.Series()
    testSummary["testId"] = int(data["TestId"][0])
    testSummary["userId"] = int(data["UserId"][0])
    testSummary["status"] = data["Status"][0]

    testSummary["totalTimeTaken"] = (data["CompletedOn"][0] -
                                     data["StartedOn"][0]).seconds

# Query the TestUserQuestion Table
if "Query the TestUserQuestion Table":
    query = """
    SELECT  
    [_QuestionId]
    ,[_QuestionTypeId]
    ,[_TestSectionId]
    ,[StartedOn]
    ,[CompletedOn]
    ,[TimeTakenInSec]
    ,[IsCorrect]
    ,[_IsAttempted]
    ,[IsMarkedForReview]
    FROM    
    [speedlabs-anon].[dbo].[TestUserQuestion]
    WHERE
    [TestUserId] = {x}
    """.format(x=testUserId)
    data = pd.read_sql(query, connection.connection)
    questionType = {1: "Obj",
                    2: "Sub",
                    3: "MCQ",
                    4: "Num",
                    5: "AnR"
                    }

    testQuestions = pd.Series()
    testQuestions["questionIds"] = data["_QuestionId"]
    testSummary["totalNoQuestions"] = len(testQuestions["questionIds"])
    testQuestions["questionTypes"] = [
        questionType[x] for x in data["_QuestionTypeId"]]

    testQuestions["questionTime"] = (
        data["CompletedOn"] - data["StartedOn"]).map(lambda x: x.seconds)
    testQuestions["questionIsCorrect"] = data["IsCorrect"]
    testQuestions["questionIsAttempted"] = data["_IsAttempted"]
    testQuestions["questionIsMarkedForReview"] = data["IsMarkedForReview"]

# Query the Question Table
if "Query the Question Table":
    query = """
    SELECT 
        [QuestionLevelId]
        ,[ChapterId]
        ,[_TopicId]
        ,[_SubjectId]
        ,[ImportanceLevel]
        ,[IsKSCQuizQuestion]
        ,[_KSCCount]
        ,[_QuestionScore]
        ,[_AverageTimeTakenInSec]
        ,[_AverageTimeTakenInSecCorrectQuestions]
        ,[_QuestionVelocity]
    FROM [speedlabs-anon].[dbo].[Question]
    WHERE [QuestionId] """
    if len(testQuestions["questionIds"]==1):
        query = query + "= {x}".format(x = str(testQuestions["questionIds"][0]))
    else:
        query = query + "IN ({x} )".format(x = ", ".join([str(i) for i in testQuestions["questionIds"]]))
    data = pd.read_sql(query, connection.connection)
    
    testSummary["chapterIds"] = data["ChapterId"].unique()
    testSummary["topicIds"] = data["_TopicId"].unique()
    testSummary["subjectIds"] = data["_SubjectId"].unique()
    testQuestions["questionScore"] = data["_QuestionScore"]
    testQuestions["averageTimeTakenInSec"] = data["_AverageTimeTakenInSec"]
    testQuestions["averageTimeTakenInSecCorrectQuestions"] = data["_AverageTimeTakenInSecCorrectQuestions"]
    testQuestions["questionVelocity"] = data["_QuestionVelocity"]

# Student Analysis
if "Student Analysis":
    # Student Attempted
    testSummary["attemptPercentage"] = 100.0 * np.sum(testQuestions["questionIsAttempted"])/testSummary["totalNoQuestions"]
    # Student Accuracy
    testSummary["accuracy"] = 100.0 * np.sum(testQuestions["questionIsCorrect"])/testSummary["totalNoQuestions"]
# testSummary
# testQuestions


NameError: name 'connection' is not defined

In [ ]:
# Get User Test Sessions:
userId = 30627
dateOnwards = '2021-1-01'
query = """
SELECT 
    [UserTestSessionId]
    ,[TopicId]
    ,[DifficultyLevelId]
    ,[_TotalQuestions]
    ,[_TotalAttempted]
    ,[_TotalCorrect]
    FROM [speedlabs-anon].[dbo].[UserTestSession]
    WHERE 
        [UserId] = {x}
        AND
        [EndedOn] > CONVERT(datetime, '{y}')
""".format(x = userId,y = dateOnwards)
data = pd.read_sql(query,connection.connection)
userSessionData = data

In [ ]:
# Get questions
userSessionId = userSessionData["UserTestSessionId"][0]

def get_session_questions(userSessionID, connection):
    query = """
    SELECT
        [QuestionId]
        ,[IsAttempted]
        ,[IsKSCAttempted]
        ,[IsCorrectlyAnswered]
        ,[TimeTakenTillSubmission]
        ,[TotalTimeTaken]
        ,[IsCorrectlyAnsweredSecondTime]
        ,[AttemptedOnSecondTime]
        ,[MarkedForRevision]
    FROM 
        [speedlabs-anon].[dbo].[UserTestSessionQuestion]
    WHERE 
        [UserTestSessionId] = {x}
    """.format(x=userSessionID)
    data = pd.read_sql(query,connection)
    return data
sessionQuestions = get_session_questions(userSessionId,connection.connection)

In [ ]:
# Generate Question Stats:

def gen_session_student_metrics(questionData):
    sessionStatisticsCol = [
        "Time",
        "NumberOfQuestions",
        "Accuracy",
        "AccuracySecond",
        "Attempted",        # Percentage
        "MarkedForReview",  # Percentage
        "ViewedKSC"         # Percentage
    ]

    sessionStats = pd.Series(index=sessionStatisticsCol)
    sessionStats["Time"] = np.sum(questionData["TimeTakenTillSubmission"])
    sessionStats["NumberOfQuestions"] = questionData["QuestionId"].shape[0]
    sessionStats["Accuracy"] = 100.0 * np.sum(questionData["IsCorrectlyAnswered"])/np.sum(1-questionData["IsCorrectlyAnswered"].isnull())
    sessionStats["AccuracySecond"] = 100.0 * np.sum(questionData["IsCorrectlyAnsweredSecondTime"])/(np.sum(1 - questionData["IsCorrectlyAnsweredSecondTime"].isnull()))
    sessionStats["Attempted"] = 100.0 * np.sum(1-questionData["IsCorrectlyAnswered"].isnull()) /questionData["QuestionId"].shape[0]
    sessionStats["MarkedForReview"] = 100.0 * np.sum(questionData["MarkedForRevision"])/questionData["QuestionId"].shape[0]
    sessionStats["ViewedKSC"] = 100.0 * np.sum(questionData["IsKSCAttempted"])/questionData["QuestionId"].shape[0]
    return sessionStats

def gen_session_global_metrics(questionIds, dateOnwards, connection):
    # Get Per Question Data
    perQuestionData = []
    for questionId in questionIds:
        query = """
        SELECT
            [QuestionId]
            ,[IsAttempted]
            ,[IsKSCAttempted]
            ,[IsCorrectlyAnswered]
            ,[TimeTakenTillSubmission]
            ,[TotalTimeTaken]
            ,[IsCorrectlyAnsweredSecondTime]
            ,[AttemptedOnSecondTime]
            ,[MarkedForRevision]
        FROM 
            [speedlabs-anon].[dbo].[UserTestSessionQuestion]
        WHERE 
            [QuestionId] = {x} AND
            [CompletedOn] > CONVERT(datetime, '{y}')
        """.format(x=str(questionId), y=dateOnwards)
        data = pd.read_sql(query, connection)
        perQuestionData.append(data)

    columns = ["QuestionId",
               "TimesViewed",
               "TimesAttempted",
               "TimesAttemptedSecond",
               "TimesKSCAttempted",
               "TimesMarkedForRevision",
               "FirstAccuracy",
               "SecondAccuracy",
               "AverageTimeTaken",
               "StdTimeTaken"]

    # Generate Global Question Statistics
    globalQuestionStats = pd.DataFrame(columns=columns)

    for question in perQuestionData:
        timesViewed = len(question)
        timesAttempted = np.sum(question["IsAttempted"])
        timesAttemptedSecond = timesViewed - \
            np.sum(question["IsCorrectlyAnsweredSecondTime"].isnull())
        timesKSCAttempted = np.sum(question["IsKSCAttempted"])
        timesIsCorrectlyAnswered = np.sum(question["IsCorrectlyAnswered"])
        timesIsCorrectlyAnsweredSecond = np.sum(
            question["IsCorrectlyAnsweredSecondTime"])
        timesMarkedForReveision = np.sum(question["MarkedForRevision"])
        averageTimeTaken = np.mean(question["TimeTakenTillSubmission"])

        questionStats = {}
        questionStats["QuestionId"] = question["QuestionId"][0]
        questionStats["TimesViewed"] = timesViewed
        questionStats["TimesAttempted"] = timesAttempted
        questionStats["TimesAttemptedSecond"] = timesAttemptedSecond
        questionStats["TimesKSCAttempted"] = timesKSCAttempted
        questionStats["TimesMarkedForRevision"] = timesMarkedForReveision
        questionStats["FirstAccuracy"] = 100.0 * \
            timesIsCorrectlyAnswered/timesAttempted
        questionStats["SecondAccuracy"] = 100.0 * \
            timesIsCorrectlyAnsweredSecond/timesAttemptedSecond
        questionStats["AverageTimeTaken"] = averageTimeTaken
        questionStats["StdTimeTaken"] = np.std(
            question["TimeTakenTillSubmission"])
        globalQuestionStats = globalQuestionStats.append(
            questionStats, ignore_index=True)

    # Generate Global Session Statistics

    sessionStatisticsCol = [
        "AverageTime",
        "StdTime",
        "Accuracy",
        "AccuracySecond",
        "Attempted",        # Percentage
        "MarkedForReview",  # Percentage
        "ViewedKSC"         # Percentage
    ]
    globalSessionStats = pd.Series()

    globalSessionStats["AverageTime"] = np.sum(globalQuestionStats["AverageTimeTaken"])
    globalSessionStats["StdTime"] = np.sqrt(np.sum(np.square(globalQuestionStats["StdTimeTaken"])))
    globalSessionStats["AvgAccuracy"] = np.mean(globalQuestionStats["FirstAccuracy"])
    globalSessionStats["StdAccuracy"] = np.std(globalQuestionStats["FirstAccuracy"])
    globalSessionStats["AvgAccuracySecond"] = np.mean(globalQuestionStats["SecondAccuracy"])
    globalSessionStats["StdAccuracySecond"] = np.std(globalQuestionStats["SecondAccuracy"])
    globalSessionStats["AvgAttempted"] = 100.0*np.mean(np.divide(globalQuestionStats["TimesAttempted"],globalQuestionStats["TimesViewed"]))
    globalSessionStats["StdAttempted"] = 100.0*np.std(np.divide(globalQuestionStats["TimesAttempted"],globalQuestionStats["TimesViewed"]))
    globalSessionStats["AvgMarkedForReview"] = 100.0*np.mean(np.divide(globalQuestionStats["TimesMarkedForRevision"],globalQuestionStats["TimesViewed"]))
    globalSessionStats["StdMarkedForReview"] = 100.0*np.std(np.divide(globalQuestionStats["TimesMarkedForRevision"],globalQuestionStats["TimesViewed"]))
    globalSessionStats["AvgViewedKSC"] = 100.0*np.mean(np.divide(globalQuestionStats["TimesKSCAttempted"],globalQuestionStats["TimesViewed"]))
    globalSessionStats["StdViewedKSC"] = 100.0*np.std(np.divide(globalQuestionStats["TimesKSCAttempted"],globalQuestionStats["TimesViewed"]))

    return perQuestionData, globalQuestionStats, globalSessionStats

def get_topic_global_metrics(topicId, dateOnwards, connection):
    query="""
    SELECT 
        [UserTestSessionId]
        ,[_TotalQuestions]
        ,[_TotalAttempted]
        ,[_TotalCorrect]
        ,[_TotalTimeTaken]
        ,[LastStartedOn]
        ,[LastEndedOn]
    FROM 
        [speedlabs-anon].[dbo].[UserTestSession] 
    WHERE
        [TopicId] = {x} AND
        [EndedOn] > CONVERT(datetime, '{y}')
    """.format(x=str(topicId), y=dateOnwards)
    topicData = pd.read_sql(query,connection)

    topicStats = pd.Series()
    topicStats["AvgNoOfQuestions"] = np.mean(topicData["_TotalQuestions"])
    topicStats["StdNoOfQuestions"] = np.std(topicData["_TotalQuestions"])
    topicStats["AvgTime"] = np.mean(topicData["_TotalTimeTaken"])
    topicStats["StdTime"] = np.std(topicData["_TotalTimeTaken"])
    topicStats["AvgAccuracy"] = 100*np.mean(np.divide(topicData["_TotalCorrect"],topicData["_TotalAttempted"]))
    topicStats["StdAccuracy"] = 100*np.std(np.divide(topicData["_TotalCorrect"],topicData["_TotalAttempted"]))
    topicStats["AvgAccuracySecond"] = None
    topicStats["StdAccuracySecond"] = None
    topicStats["AvgAttempted"] = np.mean(topicData["_TotalAttempted"])
    topicStats["StdAttempted"] = np.std(topicData["_TotalAttempted"])
    topicStats["AvgViewedKSC"] = None
    topicStats["StdViewedKSC"] = None
    return topicStats


In [ ]:
a,b,c = gen_session_global_metrics(sessionQuestions["QuestionId"],dateOnwards,connection.connection)
d = gen_session_student_metrics(sessionQuestions)
e = get_topic_global_metrics(userSessionData["TopicId"][0],dateOnwards,connection.connection)

In [ ]:
print("Pseudo Session Metrics:")
c

In [ ]:
print("Topic Metrics:")
e

In [ ]:
print("Student Metrics:")
d

In [ ]:
b["TimesAttempted"].sum()